In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

from math import exp
from math import log
from collections import defaultdict
import time
import sys

### KNN

In [2]:
def load_data(file_dir):

    user_ids_dict, rated_item_ids_dict = {},{}
    N, M, u_idx, i_idx = 0,0,0,0 
    data = []
    f = open(file_dir)
    for line in f.readlines():
        if '::' in line:
            u, i, r, _ = line.split('::')
        else:
            u, i, r, _ = line.split()
    
        if int(u) not in user_ids_dict:
            user_ids_dict[int(u)]=u_idx
            u_idx+=1
        if int(i) not in rated_item_ids_dict:
            rated_item_ids_dict[int(i)]=i_idx
            i_idx+=1
        data.append([user_ids_dict[int(u)],rated_item_ids_dict[int(i)],float(r)])
    
    f.close()
    N = u_idx
    M = i_idx

    return N, M, data, rated_item_ids_dict


def sequence2mat(sequence, N, M):
    
    records_array = np.array(sequence)
    mat = np.zeros([N,M])
    row = records_array[:,0].astype(int)
    col = records_array[:,1].astype(int)
    values = records_array[:,2].astype(np.float32)
    mat[row,col]=values
    
    return mat


def calculate_similarity(a, b, model='pearson', minimum_common_items=5):
    assert a.shape==b.shape
    dim = len(a.shape) #向量维度
    common_items = a*b>0 # 共同评分的项
    common_size = np.sum(common_items,axis=dim-1)
    
    if model=='pearson':
        mean_a = np.sum(a,axis=dim-1)/np.sum(a>0,axis=dim-1)
        mean_b = np.sum(b,axis=dim-1)/np.sum(b>0,axis=dim-1)
        if dim ==1: #若是两个列向量
            aa = (a - mean_a)*common_items
            bb = (b - mean_b)*common_items
        else:
            aa = (a - np.reshape(mean_a, (-1,1)))*common_items
            bb = (b - np.reshape(mean_b, (-1,1)))*common_items
    else: #consine
        mean_u = np.sum(b,axis=0)/np.sum(b>0,axis=0)
        aa = (a - mean_u)*common_items
        bb = (b - mean_u)*common_items
        
    sim = np.sum(aa*bb, axis=dim-1)/(np.sqrt(np.sum(aa**2, axis=dim-1))*np.sqrt(np.sum(bb**2, axis=dim-1)) + 1e-10)
    least_common_items = common_size>minimum_common_items # 共同评分的商品不少于least_common_items
    return sim*least_common_items


def similarity_matrix(mat, model='pearson', minimum_common_items=5):
    n,m = mat.shape
    sim_list=[]
    for u in range(n):
        a = np.tile(mat[u,:], (n,1))
        b = mat
        if model=='pearson':
            sim = calculate_similarity(a, b, model='pearson', minimum_common_items=minimum_common_items)
        else: # consine
            sim = calculate_similarity(a, b, model='consine', minimum_common_items=minimum_common_items)
        sim_list.append(sim)
        if u % 100 ==0:
            print(u)
    return np.array(sim_list)

In [3]:
file_dir = "./ml-1m/ratings.dat"      # "./ml-100k/u.data"

N, M, data_list, item_ids_dict = load_data(file_dir=file_dir )
print(' data length: %d \n user number: %d \n item number: %d' %(len(data_list),N,M))

 data length: 1000209 
 user number: 6040 
 item number: 3706


In [4]:
train_list, test_list = train_test_split(data_list,test_size=0.2)

train_mat = sequence2mat(sequence = train_list, N = N, M = M)
test_mat = sequence2mat(sequence = test_list, N = N, M = M)

sim_mat_pearson = similarity_matrix(mat=train_mat, model='pearson')
neighbors_pearson = np.argsort(-np.array(sim_mat_pearson)) # 获取近邻
sim_sort_pearson = -1*np.sort(-np.array(sim_mat_pearson)) # 获取对应近邻的相似度
sim_mat_normal = similarity_matrix(mat=train_mat, model='123645')
neighbors_normal = np.argsort(-np.array(sim_mat_normal)) # 获取近邻
sim_sort_normal = -1*np.sort(-np.array(sim_mat_normal)) # 获取对应近邻的相似度

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000


C:\Users\32698\AppData\Local\Temp\ipykernel_2188\2262155707.py:56: RuntimeWarning: invalid value encountered in true_divide
  mean_u = np.sum(b,axis=0)/np.sum(b>0,axis=0)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000


In [5]:
def get_K(sim_mat, min_similarity=0.5):
    num = np.sum(sim_mat[:,1:]>min_similarity, axis=1)     
    num_sort = np.sort(-num)
    line = int(0.8*len(sim_mat))
    K = -1*num_sort[line]
    return K

def prediction(train_mat, sim_mat, K=1, model='user_based'):
    assert len(train_mat.shape)>1
    n,m = train_mat.shape
    
    if  model=='user_based':
        sim_sort = -1*np.sort(-np.array(sim_mat))[:,1:K+1] # 除去最相似的自己
        neighbors = np.argsort(-np.array(sim_mat))[:,1:K+1]
        common_items = train_mat[neighbors]>0 
        mean_user = np.reshape(np.sum(train_mat,axis=1)/np.sum(train_mat>0,axis=1), (-1,1))
        mat_m = train_mat - mean_user
        aa = np.sum(sim_sort[:,:,np.newaxis]*mat_m[neighbors]*common_items,axis=1)
        bb = np.sum(sim_sort[:,:,np.newaxis]*common_items,axis=1)+1e-10 # 1e-10保证分母不为０
        r_pred = mean_user + aa/bb
        return r_pred
    else: # 'item_based'
        r_pred=[]
        for u in range(n):
            u_mat = np.tile(train_mat[u],(m,1)) # m份用户u的记录,m*m
            rated_items_sim = (u_mat>0)*sim_mat # 保留有评分记录的相似度 m*m
            sim_sort = -1*np.sort(-np.array(rated_items_sim))[:,:K] # m*K
            neighbors = np.argsort(-np.array(rated_items_sim))[:,:K] # m*K
            neighbor_ratings = np.array([u_mat[i,neighbors[i]] for i in range(m)])# m*K
            aa = np.sum(sim_sort*neighbor_ratings,axis=1) # m*1
            bb = np.sum(sim_sort,axis=1)+1e-10 # 1e-10保证分母不为０ m*1
            r_pred.append(aa/bb)
        
        return np.array(r_pred)  

In [6]:
K=5
r_pred = prediction(train_mat=train_mat, sim_mat=sim_mat_pearson, K=K, model='user_based')
r_pred_cos = prediction(train_mat=train_mat, sim_mat=sim_mat_normal, K=K, model='user_based')

In [7]:
def get_dcg(ranklist, relevant_items):
    dcg = 0.0
    for i, item in enumerate(ranklist):
        if item in relevant_items:
            dcg += 1 / math.log2(i + 2)
    return dcg

def get_idcg(relevant_items):
    idcg = 0.0
    for i in range(len(relevant_items)):
        idcg += 1 / math.log2(i + 2)
    return idcg

def get_ndcg(ranklist, relevant_items):
    dcg = get_dcg(ranklist, relevant_items)
    idcg = get_idcg(relevant_items)
    return dcg / idcg if idcg > 0 else 0

def calculate_ndcg_for_arrays(ranklists, relevant_items_sets):
    ndcgs = []
    for ranklist, relevant_items in zip(ranklists, relevant_items_sets):
        ndcgs.append(get_ndcg(ranklist, set(relevant_items)))
    return ndcgs

def recall_precision(topn, test_mat):
    n,m = test_mat.shape
    hits,total_pred,total_true = 0.,0.,0.
    for u in range(n):
        hits += len([i for i in topn[u,:] if test_mat[u,i]>0])
        size_pred = len(topn[u,:])
        size_true = np.sum(test_mat[u,:]>0,axis=0)
        total_pred += size_pred
        total_true += size_true

    recall = hits/total_true
    precision = hits/total_pred
    return recall, precision
def get_topn(r_pred, train_mat, n=10):
    unrated_items = r_pred * (train_mat==0)
    idx = np.argsort(-unrated_items)
    return idx[:,:n]
topn = get_topn(r_pred=r_pred, train_mat=train_mat, n=5)
topncos = get_topn(r_pred=r_pred_cos, train_mat=train_mat, n=5)
recall, precision = recall_precision(topn=topn, test_mat=test_mat)
print('Pearson recall:%.4f; precision:%.4f'%(recall,precision)) 
recall, precision = recall_precision(topn=topn, test_mat=test_mat)
print('Cosine recall:%.4f; precision:%.4f'%(recall,precision))


Pearson recall:0.0077; precision:0.0510
Cosine recall:0.0077; precision:0.0510


In [8]:
ndcg = calculate_ndcg_for_arrays(topn, test_mat)
ndcg_=np.mean(ndcg)
ndcg_

0.013623504950467507

### BPR-KNN

In [2]:
class KNN(object):

    def __init__(self, numUsers, numItems, lamI = 6e-2, lamJ = 6e-3, learningRate = 0.1):
        self._numUsers = numUsers
        self._numItems = numItems
        self._lamI = lamI
        self._lamJ = lamJ
        self._learningRate = learningRate
        self._users = set()
        self._items = set()
        self._Iu = defaultdict(set)
        
        
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def train(self, trainData, epochs=30, batchSize=500):
        
        # correlation matrix
        self.C =np.random.rand(self._numItems, self._numItems)  
        for l in range(self._numItems):
            self.C[l][l] = 0
            for n in range(l, self._numItems):
                self.C[l][n] = self.C[n][l]
              
        # change batch_size to min(batch-size, len(train))
        if len(trainData) < batchSize:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(trainData)))
            batchSize = len(trainData)
                  
        self._trainDict, self._users, self._items = self._dataPretreatment(trainData)
        N = len(trainData) * epochs
        users, pItems, nItems = self._sampling(N)
        itr = 0
        t2 = t0 = time.time()
        while (itr+1)*batchSize < N:
      
            self._mbgd(
                users[itr*batchSize: (itr+1)*batchSize],
                pItems[itr*batchSize: (itr+1)*batchSize],
                nItems[itr*batchSize: (itr+1)*batchSize]
            )
            
            itr += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.3f%% ) in %.1f seconds" %(str(itr*batchSize), 100.0 * float(itr*batchSize)/N, t2 - t0))
            sys.stderr.flush()
        if N > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %.2f samples per second\n" % (t2 - t0, N*1.0/(t2 - t0)))
            sys.stderr.flush()
            
            
    def _mbgd(self, users, pItems, nItems):
        
        prev = -2**10
        for _ in range(30):
            
            gradientC = defaultdict(float)
            obj = 0

            for ind in range(len(users)):
                u, i, j = users[ind], pItems[ind], nItems[ind]
                x_ui = sum([self.C[i][l] for l in self._Iu[u] if i != l])
                x_uj = sum([self.C[j][l] for l in self._Iu[u]])
                x_uij =  x_ui - x_uj
                
                for l in self._Iu[u]:
                    if l != i:
                        gradientC[(i,l)] += (1-self.sigmoid(x_uij)) + self._lamI * self.C[i][l]**2
                        gradientC[(l,i)] += (1-self.sigmoid(x_uij)) + self._lamI * self.C[l][i]**2
                    gradientC[(j,l)] += -(1-self.sigmoid(x_uij)) + self._lamJ * self.C[j][l]**2
                    gradientC[(l,j)] += -(1-self.sigmoid(x_uij)) + self._lamJ * self.C[l][j]**2
                    
                    obj -= 2*self._lamI * self.C[i][l]**2 + 2*self._lamJ * self.C[j][l]**2
                    
                obj += log(self.sigmoid(x_uij))
            
            #print 'OBJ: ', obj
            if prev > obj: 
                break
            prev = obj
            
            for a,b in gradientC:
                self.C[a][b] += self._learningRate * gradientC[(a,b)]
            
        #print _, '\n'
        
    def _sampling(self, N):
        
        sys.stderr.write("Generating %s random training samples\n" % str(N))
        userList = list(self._users)
        userIndex = np.random.randint(0, len(self._users), N)
        pItems, nItems = [], []
        cnt = 0
        for index in userIndex:
            u = userList[index]
            i = self._trainDict[u][np.random.randint(len(self._Iu[u]))]
            pItems.append(i)
            j = np.random.randint(self._numItems)
            while j in self._Iu[u]:
                j = np.random.randint(self._numItems)
            nItems.append(j)
            
            cnt += 1
            if not cnt %10000:
                sys.stderr.write("\rGenerated %s" %(str(cnt)))
                sys.stderr.flush()
        return userIndex, pItems, nItems

    def predictionsKNN(self, K, u):
        #slow
        if K >= len(self._Iu[u]):
            res = np.sum([self.C[:,l] for l in self._Iu[u]], 0)
        else:
            res = []
            for i in range(self._numItems):
                res.append(sum(sorted([self.C[i][l] for l in self._Iu[u]], reverse=True)[:K]))
        return res

    def predictionsAll(self, u):
        
        res = np.sum([self.C[:,l] for l in self._Iu[u]], 0)
        return res

    def prediction(self, u, i):
        
        scores = self.predictionsAll(u)
        return scores[i] > sorted(scores)[self._numItem*0.8]

    def _dataPretreatment(self, data):
        dataDict = defaultdict(list)
        items = set()
        for u, i in data:
            self._Iu[u].add(i)
            dataDict[u].append(i)
            items.add(i)
        return dataDict, set(dataDict.keys()), items

In [3]:
loss=[]
prec=[]
recall=[]
ndcg=[]

class BPR(nn.Module):
    def __init__(self):
        super(BPR, self).__init__()
        self.W = None             # user matrix
        self.H = None             # item matrix
        
        self.uid = None
        self.iid = None
        
        # 用户u对应他访问过的所有items集合
        self.train_user_items = None
        self.test_user_items = None
        
        # (u, i, rating) dataset
        self.train = None
        self.test = None
        
    def _split(self, df, ratio):
        train = pd.DataFrame(columns = df.columns, dtype=int)
        test = pd.DataFrame(columns = df.columns, dtype=int)
        for i in self.uid:
            train_1, test_1 = train_test_split(df[df.iloc[:, 0] == i], train_size = ratio, shuffle = True, random_state = 5)
            train = pd.concat([train, train_1])
            test = pd.concat([test, test_1])
        return train, test    
    
    def preprocess(self, df, train_size=0.8):
        df = df.rename(columns = {df.columns[0]: 'ori_uid', df.columns[1]: 'ori_iid', df.columns[2]: 'rating'})
        df = df.groupby('ori_uid').filter(lambda x: x['ori_uid'].count()>=10)
        uid_map = pd.DataFrame({"ori_uid": np.asarray(list(set(df.iloc[:,0].values)))})
        uid_map["serial_uid"] = uid_map.index
        iid_map = pd.DataFrame({"ori_iid": np.asarray(list(set(df.iloc[:,1].values)))})
        iid_map["serial_iid"] = iid_map.index
        
        self.uid = uid_map["serial_uid"].values
        self.iid = iid_map["serial_iid"].values
        
        df = df.merge(uid_map, left_on = 'ori_uid', right_on = 'ori_uid', how="left")
        df = df.merge(iid_map, left_on = 'ori_iid', right_on = 'ori_iid', how="left")
        df = df[['serial_uid', 'serial_iid', 'rating']]
        
        train, test = self._split(df, train_size)
        
        self.train_user_items = train.groupby(train.columns[0])[train.columns[1]].apply(lambda x: list(x)).to_list()
        self.test_user_items = test.groupby(test.columns[0])[test.columns[1]].apply(lambda x: list(x)).to_list()
        
        self.train = train
        self.test = test
        return self.train_user_items, self.test_user_items, self.iid, self.uid
    
    def generate_train_batch(self, batch, sets,ite=10):
        train = []
        for b in range(batch):
            u = self.uid[np.random.randint(0, len(self.uid))]
            i = sets[u][np.random.randint(0, len(sets[u]))]
            lis = [u,i]
            for a in range(ite):
                j = self.iid[np.random.randint(0, len(self.iid))]
                while j in sets[u]:
                    j = self.iid[np.random.randint(0, len(self.iid))]
                lis.append(j)
            train.append(lis)
        return np.asarray(train) 
    
    def fit(self, k, stepsize=0.05, max_iter=10, batch=10000, n=10,ite=10):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        self.W = nn.Parameter(torch.rand(len(self.uid), k).to(device) * 0.01)    # 初始化 W，H
        self.H = nn.Parameter(torch.rand(len(self.iid), k).to(device) * 0.01)  
        
        # 创建字典：用户u对应他访问过的所有items集合
#         self.train_user_items = df.groupby(df.columns[0])[df.columns[1]].apply(lambda x: np.array(x)).to_list()
        
        optimizer = optim.Adam([self.W, self.H], lr=stepsize)     # 主模型优化器
        for x in range(max_iter):
            #取训练批次：uij三元组
            uij = self.generate_train_batch(batch, self.train_user_items,ite)
            
            u = uij[:, 0]
            i = uij[:, 1]
            
            u_emb = self.W[u]
            i_emb = self.H[i]
            
            
            j_embs = torch.zeros_like(i_emb) 
            
            for i in range(2, 2 + ite):  
                j = uij[:, i]  
                j_emb = self.H[j]  
                j_embs+=torch.exp(u_emb*j_emb)
            j_embs=j_embs/ite
            
            
            optimizer.zero_grad()
            loss = torch.sum((torch.sum(-u_emb * i_emb +torch.log(torch.exp(u_emb * i_emb) +j_embs+1e-8))+1e-8))
            loss.backward()
            optimizer.step()
            
            print(f"Train | {x+1}/{max_iter}, BPR loss: {loss.item() / batch}")
            
            rec, pre, ndcg = self.performance(n)
            print(f'Valid | {x+1}/{max_iter}, Pre@{n}: {pre}, Recl@{n}: {rec}, NDCG@{n}: {ndcg}')
            print('---------------------------------------------------')
    
    def _predict(self, uid, items, n):
        scores = torch.mv(self.H[items], self.W[uid])
        if n > scores.shape[0]: 
            n = scores.shape[0]
        top_N_val, top_N_idx = torch.topk(scores, k=n)
        return list(zip(items[top_N_idx.cpu()], top_N_val.cpu()))

    def NDCG(self, uid, n):         # 用模型排序+真实分数计算 DCG, 重排后计算 iDCG
        test_user = self.test[self.test.iloc[:, 0] == uid]
        rating = self._predict(uid, test_user.iloc[:, 1].values, n)
        irating =sorted(test_user.iloc[:, 2].values, reverse=True)
        dcg = 0
        idcg = 0
        if n > len(irating): n = len(irating)  
        for i in range(n):
            r = test_user[test_user.iloc[:, 1]==rating[i][0]].iloc[0, 2]
            dcg += 1.0 * (2**r - 1) / math.log(i + 2, 2)
            idcg += 1.0 * (2**irating[i] - 1) / math.log(i + 2, 2)
        return dcg / idcg

    def performance(self, n):      # Output recall@n, precision@n, NDCG@n
        hit = 0
        n_recall = 0
        n_precision = 0
        ndcg = 0
        
        for i in self.uid:
            # Items that User i hasn't tried in training set
            unknown_items = np.setdiff1d(self.iid, self.train_user_items[i])
            # Items that User i actually tried in testing set
            known_items = self.test_user_items[i]
            
            #目标：预测 unknown items 中的top_N，若击中test中的items，则为有效预测
            ru = self._predict(i, unknown_items, n)
            
            hit += sum(1 for item, pui in ru if item in known_items)
            n_recall += len(known_items)
            n_precision += n
            ndcg += self.NDCG(i, n)
            
        recall = hit / (1.0 * n_recall)
        precision = hit / (1.0 * n_precision)
        ndcg /= len(self.uid)
        return recall, precision, ndcg

In [4]:
df1 = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=['user id', 'item id', 'rating', 'timestamp'], engine='python')

In [5]:
model1 = BPR()
train_user_items, test_user_items,iid, uid=model1.preprocess(df1)
train = model1.train
test = model1.test
train1 = [tuple(i) for i in train.iloc[:, :2].values.tolist()]
test1 = [tuple(i) for i in test.iloc[:, :2].values.tolist()]
u = len(model1.uid)
i = len(model1.iid)

In [ ]:
AKNN=KNN(u,i)
AKNN.train(train1,20,128)

Generating 15955160 random training samples
Processed 100000 ( 0.627% ) in 296.0 seconds

In [ ]:
aknn=[]
for i in uid:
    aknn.append(AKNN.predictionsKNN(5, i))
aknnt= torch.tensor(aknn)

In [ ]:
def NDCG(uid, n):         # 用模型排序+真实分数计算 DCG, 重排后计算 iDCG
    test_user = test[test.iloc[:, 0] == uid]
    rating = predict(uid, test_user.iloc[:, 1].values, n)
    irating =sorted(test_user.iloc[:, 2].values, reverse=True)
    dcg = 0
    idcg = 0
    if n > len(irating): n = len(irating)  
    for i in range(n):
        r = test_user[test_user.iloc[:, 1]==rating[i][0]].iloc[0, 2]
        dcg += 1.0 * (2**r - 1) / math.log(i + 2, 2)
        idcg += 1.0 * (2**irating[i] - 1) / math.log(i + 2, 2)
    return dcg / idcg

def predict( uid, items, n):
    scores =aknnt[uid][items]
    if n > scores.shape[0]: 
        n = scores.shape[0]
    top_N_val, top_N_idx = torch.topk(scores, k=n)
    return list(zip(items[top_N_idx.cpu()], top_N_val.cpu())) 

def performanceout(n):
    hit = 0
    n_recall = 0
    n_precision = 0
    ndcg = 0
    for i in uid:
        unknown_items = np.setdiff1d(iid, train_user_items[i])

        known_items = test_user_items[i]

        ru = predict(i, unknown_items, n)
        
        hit += sum(1 for item, pui in ru if item in known_items)
        n_recall += len(known_items)
        n_precision += n
        ndcg += NDCG(i, n)

    recall = hit / (1.0 * n_recall)
    precision = hit / (1.0 * n_precision)
    ndcg /= len(uid)
    return recall, precision, ndcg    
    
performanceout(10)    

In [ ]:
rec, pre, ndcg = performanceout(10) 
print(f' Pre@10: {pre}, Rec@10: {rec}, NDCG@10: {ndcg}')